# <center>3.4.1 - Regularization</center>

<center> <img src = https://cdn-images-1.medium.com/max/1000/0*7tki_O81VJx7oCnA> </center>

Лассо- и ридж-регрессии. [Источник](https://hastie.su.domains/Papers/ESLII.pdf)

Для победы в соревнованиях по машинному обучению вы редко будете использовать линейную регрессию, но на этой простой модели легко показать принципы регуляризации и отбора признаков, которые в той или иной форме встречаются в других более сложных алгоритмах. Хорошо разобравшись в этой теме, уже можно обучать устойчивые модели, которые не будут улетать вниз на прайват части лидерборда, а так же эти знания пригодятся и в продакшен моделях, которые не будут ломаться от каждого изменения в данных.<br>
Постараемся в этом ноутбуке обойтись без формул и доказательств, а сразу посмотреть всё на примерах.

У модели два основных источника ошибок:
* **Дисперсия** (variance) - ошибка, связанная с чувствительностью к малейшим изменениям в обучающих данных.
* **Смещение** (bias) - ошибка, связанная с неверными предположениями модели относительно связи признаков и таргета.

Если сумма этих ошибок будет минимальной, то получим на выходе максимально эффективную устойчивую модель.

<center> <img src = https://www.researchgate.net/publication/342624204/figure/fig7/AS:908763229847559@1593677446952/overfitting-vs-underfitting.ppm> </center>

Рассмотрим формулу линейной регрессии:
* Yi - зависимая переменная (таргет)
* Xi - независимая переменная (признак, предиктор)
* β - коэффициенты, при домножении X на которые получаем искомый прогноз, причем β0 - свободный член (константа, intercept), равен 0, если функция проходит через начало координат. 

<center> <img src = https://www.isixsigma.com/wp-content/uploads/2018/11/linear_regression_transfer_function.png height=400 width=500> </center>

### Обычно задача регрессии формулируется так:
Имея набор признаков (обычно матрица X), требуется найти набор коэффициентов (обычно вектор β), котрые нужно умножить на значения Х, чтобы получить предсказание (обычно вектор Y).

<center> <img src = https://cdn-images-1.medium.com/max/1000/0*a7N1cvDY0pN9CAKp height=500 width=700> </center>

Довольно часто при обучении регрессии происходит "черезмерно близкая подгонка" данных (оверфит). В этом случае нам помогут методы регуляризации.

## Ридж- и Лассо-регрессии
Ридж и Лассо (Ridge & Lasso)   —  это модели линейной регрессии, но с поправочными (штрафными) коэффициентами, также называемыми регуляризацией. Они вносят поправки в размерность β-вектора разными способами.
* **Лассо-регрессия (l1-регуляризация)** — накладывает штраф на l1-норму β-вектора. l1-норма вектора  —  это сумма абсолютных значений в этом векторе. Модель пытается достичь большей точности, путем нахождения и отбрасывания бесполезных коэффициентов.
* **Ридж-регрессия (l2-регуляризация)** — накладывает штраф на l2-норму β-вектора. l2-норма вектора  —  это квадратный корень из суммы квадратов значений в векторе. Ридж-регрессия не позволяет коэффициентам β-вектора достигать экстремальных значений (что часто происходит при переобучении). Модель пытается достичь большей точности, при этом ни один коэффициент не должен достигать экстремальных значений.

Оба эти метода имеют **коэффициент регуляризации** (называемый “лямбда”), который контролирует величину штрафа. При λ=0 как лассо-, так и ридж-регрессия становятся моделями линейной регрессии (в этом случае просто не накладываются никакие штрафы). При увеличении лямбды возрастает ограничение на размер β-вектора. При этом каждая регрессия оптимизирует его по-своему, пытаясь подобрать наилучший набор коэффициентов с учетом собственных ограничений.<br>
Переходим от теории к практике.

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import mean_squared_error

In [99]:
# Загружаем train-датасет который мы сохранили на шаге quickstart
rides_info = pd.read_csv('../data/car_train.csv')
rides_info.head()

,car_id,model,car_type,fuel_type,car_rating,year_to_start,riders,year_to_work,target_reg,target_class
0,y13744087j,Kia Rio X-line,economy,petrol,3.78,2015,76163,2021,9492.96,another_bug
1,O41613818T,VW Polo VI,economy,petrol,3.90,2015,78218,2021,2656.23,electro_bug
2,d-2109686j,Renault Sandero,standart,petrol,6.30,2012,23340,2017,1526.11,gear_stick
3,u29695600e,Mercedes-Benz GLC,business,petrol,4.04,2011,1263,2020,1338.00,engine_fuel
4,N-8915870N,Renault Sandero,standart,petrol,4.70,2012,26428,2017,825.72,engine_fuel


In [100]:
drop_cols = ['car_id', 'target_reg', 'target_class']
cat_cols = ['car_type', 'fuel_type', 'model']

In [101]:
# закодируем категориальные фичи в one hot encoding вектора
rides_info = pd.get_dummies(rides_info, columns=cat_cols)

Так же важно отметить, что признаки следует отнормировать перед подачей в лиейную модель. Так же без нормировки не будет работать регуляризация.

In [102]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
target_scaler = StandardScaler()
df = scaler.fit_transform(rides_info.drop(drop_cols, axis=1))
target = target_scaler.fit_transform(rides_info['target_reg'].values.reshape(-1, 1))

In [103]:
X = df
y = rides_info['target_reg']

In [104]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Linear regression

In [105]:
linear_reg = LinearRegression()
linear_reg.fit(X_train, y_train)

y_pred = linear_reg.predict(X_test)
#y_pred = target_scaler.inverse_transform(np.array(y_pred).reshape(-1, 1))
#y_true = target_scaler.inverse_transform(np.array(y_test).reshape(-1, 1))
mse_linear = mean_squared_error(y_pred, y_test)

print(mse_linear)

22521314.889481395


In [69]:
print(linear_reg.coef_)

[[ 5.01342635e-03  9.70788289e-02 -9.54962557e-02 -8.72597465e-04
  -8.97037442e-02 -4.03444738e-02  8.73116771e-03  1.98885747e-01
   6.06575884e-03 -6.30693969e+11  1.09766504e-01  3.20758074e+12
  -1.68121684e+12 -6.22780098e+12 -4.92456342e+10  1.83130125e+12
   1.12626373e+12  3.01135758e+12  3.26416340e+12  3.34405030e+12
  -1.03385169e+12 -1.09609275e+12  1.44690217e+12  1.21512471e+12
   1.36925115e+12  1.35573819e+12  5.54399791e+11 -1.65223022e+12
  -1.23767402e+12 -1.06544033e+12 -1.21073060e+12  5.52636989e+11
   5.66538674e+11  5.63105602e+11  1.38700335e+12  1.25088228e+12
   1.43434605e+12  1.30939183e+12  2.35094756e+12  1.36026182e+12
   1.40878364e+12  1.33282253e+12  2.26591731e+12]]


In [70]:
print(linear_reg.intercept_)

[0.00945674]


### Lasso Regression

In [71]:
lambda_values = [0.000001, 0.0001, 0.001, 0.005, 0.01, 0.05,  0.1, 0.2, 0.3, 0.4, 0.5]

for lambda_val in lambda_values:
    lasso_reg = Lasso(lambda_val)
    lasso_reg.fit(X_train, y_train)
    y_pred = lasso_reg.predict(X_test)
    y_pred = target_scaler.inverse_transform(np.array(y_pred).reshape(-1, 1))
    y_true = target_scaler.inverse_transform(np.array(y_test).reshape(-1, 1))
    rmse_lasso = mean_squared_error(y_pred, y_true) ** 0.5
    #mse_lasso = mean_squared_error(y_pred, y_test)
    print(("Lasso MSE with Lambda={} is {}").format(lambda_val, rmse_lasso))

/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.184e+01, tolerance: 2.011e-01
  model = cd_fast.enet_coordinate_descent(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.798e+01, tolerance: 2.011e-01
  model = cd_fast.enet_coordinate_descent(


Lasso MSE with Lambda=1e-06 is 4608.4373538552545
Lasso MSE with Lambda=0.0001 is 4606.834780701058
Lasso MSE with Lambda=0.001 is 4592.653309595803
Lasso MSE with Lambda=0.005 is 4556.644727418629
Lasso MSE with Lambda=0.01 is 4520.929298461938
Lasso MSE with Lambda=0.05 is 4358.007330100647
Lasso MSE with Lambda=0.1 is 4284.551143282346
Lasso MSE with Lambda=0.2 is 4192.366649503104
Lasso MSE with Lambda=0.3 is 4190.792337600921
Lasso MSE with Lambda=0.4 is 4417.512578501489
Lasso MSE with Lambda=0.5 is 4795.731522431299


In [58]:
print(lasso_reg.coef_)

[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00 -0.00000000e+00 -0.00000000e+00  0.00000000e+00
 -0.00000000e+00  0.00000000e+00 -0.00000000e+00  0.00000000e+00
 -0.00000000e+00  0.00000000e+00 -0.00000000e+00  1.06835227e-01
 -1.26122304e-16  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -0.00000000e+00 -0.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00 -0.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
 -0.00000000e+00 -0.00000000e+00  1.39288618e-03 -0.00000000e+00
 -0.00000000e+00 -0.00000000e+00  0.00000000e+00]


### Ridge Regression

In [59]:
lambda_values = [0.00001, 0.01, 0.05, 0.1, 0.5, 1, 1.5, 3, 5, 6, 7, 8, 9, 10]

for lambda_val in lambda_values:
    ridge_reg = Ridge(lambda_val)
    ridge_reg.fit(X_train, y_train)
    y_pred = ridge_reg.predict(X_test)
    y_pred = target_scaler.inverse_transform(np.array(y_pred).reshape(-1, 1))
    y_test = target_scaler.inverse_transform(np.array(y_test).reshape(-1, 1))
    mse_ridge = mean_squared_error(y_pred, y_test)
    print(("Lasso MSE with Lambda={} is {}").format(lambda_val, mse_ridge))

Lasso MSE with Lambda=1e-05 is 4.250223407061632e+98
Lasso MSE with Lambda=0.01 is 1.6305265608311501e+106
Lasso MSE with Lambda=0.05 is 6.255240280213595e+113
Lasso MSE with Lambda=0.1 is 2.39971748410289e+121
Lasso MSE with Lambda=0.5 is 9.206111588910005e+128
Lasso MSE with Lambda=1 is 3.531769516574863e+136
Lasso MSE with Lambda=1.5 is 1.354903837276241e+144
Lasso MSE with Lambda=3 is 5.197860165139602e+151
Lasso MSE with Lambda=5 is 1.994071428025386e+159
Lasso MSE with Lambda=6 is 7.649918877647235e+166
Lasso MSE with Lambda=7 is 2.9347624168374834e+174
Lasso MSE with Lambda=8 is 1.1258721276702869e+182
Lasso MSE with Lambda=9 is 4.3192186208748683e+189
Lasso MSE with Lambda=10 is 1.6569954115051622e+197


In [60]:
print(ridge_reg.coef_)

[[ 5.09613385e-03  6.77829545e-02 -6.51783240e-02  9.66755979e-04
  -8.86240843e-02 -4.08858967e-02  7.92649516e-03  1.96856254e-01
   6.85570271e-03  0.00000000e+00  1.02516474e-01  4.50418024e-02
  -7.09850121e-02  2.11329554e-01 -3.59478576e-02  2.00620153e-01
  -2.00620153e-01  8.27050812e-03  5.00297417e-02  2.39450013e-03
   2.41953588e-02  1.07934084e-02 -3.38993535e-02 -2.00132391e-02
  -2.47701707e-02 -2.02245377e-02 -2.75603046e-02 -5.90390846e-02
   2.61237059e-02  8.97736426e-02  2.32745617e-02 -3.90944286e-03
  -2.88020367e-02 -3.69631841e-03 -9.40380498e-03  9.66108482e-03
   4.55834387e-03 -4.60453709e-05  3.77174923e-01 -2.04952152e-02
  -7.80754556e-03 -1.83173402e-02  6.61647571e-02]]


### Elastic-Net - COMBO L1 + L2
Посмотрим описание из документации библиотеки [sklearn](https://scikit-learn.ru/1-1-linear-models/#elastic-net):
>ElasticNet is a linear regression model trained with both and -norm regularization of the coefficients. This combination allows for learning a sparse model where few of the weights are non-zero like Lasso, while still maintaining the regularization properties of Ridge. We control the convex combination of and using the l1_ratio parameter.
Elastic-net is useful when there are multiple features that are correlated with one another. Lasso is likely to pick one of these at random, while elastic-net is likely to pick both. A practical advantage of trading-off between Lasso and Ridge is that it allows Elastic-Net to inherit some of Ridge’s stability under rotation.

>ElasticNet полезна, когда есть несколько признаков, которые коррелируют друг с другом. Лассо, вероятно, выберет один из них наугад, а elastic-net — и то, и другое.

In [48]:
l1_ratios = [0.00001, 0.001, 0.01, 0.05, 0.1, 0.5, 0.7, 0.9, 1]

for l1_ratio in l1_ratios:
    elasticnet_reg = ElasticNet(l1_ratio=l1_ratio)
    elasticnet_reg.fit(X_train, y_train)
    y_pred = elasticnet_reg.predict(X_test)
    mse_ridge = mean_squared_error(y_pred, y_test)
    print(("ElasticNet MSE with l1_ratio={} is {}").format(l1_ratio, mse_ridge))

ElasticNet MSE with l1_ratio=1e-05 is 0.4105907387649885
ElasticNet MSE with l1_ratio=0.001 is 0.41089887912213874
ElasticNet MSE with l1_ratio=0.01 is 0.4145135057200774
ElasticNet MSE with l1_ratio=0.05 is 0.42863670853916197
ElasticNet MSE with l1_ratio=0.1 is 0.43920637776001514
ElasticNet MSE with l1_ratio=0.5 is 0.6148990817492882
ElasticNet MSE with l1_ratio=0.7 is 0.6959181878204748
ElasticNet MSE with l1_ratio=0.9 is 0.6959181878204748
ElasticNet MSE with l1_ratio=1 is 0.6959181878204748


In [49]:
print(elasticnet_reg.coef_)

[ 0.  0.  0.  0.  0. -0. -0.  0. -0.  0. -0.  0. -0.  0. -0.  0. -0.  0.
  0.  0.  0.  0. -0. -0. -0. -0. -0.  0.  0.  0.  0. -0. -0. -0. -0. -0.
 -0. -0.  0. -0. -0. -0.  0.]


## Что же выбрать?
* **Лассо (l1-регуляризацию)** следует использовать, когда есть несколько характеристик с высокой предсказательной способностью, а остальные бесполезны. Она обнуляет бесполезные характеристики и оставляет только подмножество переменных.

* **Ридж (l2-регуляризацию)** лучше применять, когда предсказательная способность набора данных распределена между различными характеристиками. Ридж-регрессия не обнуляет характеристики, которые могут быть полезны при составлении прогнозов, а просто уменьшает вес большинства переменных в модели.

На практике это обычно трудно определить поэтому лучше экспериментировать на тестовом множестве, используя различные значения лямбды.
### Зачем же в итоге нужна регуляризация?
* Регуляризация предназначена для регулирования сложности модели и её целью является упрощение модели.
* Регуляризация помогает бороться с переобучением и увеличивает обобщающую способность (робастность) модели.
* Регуляризация применяется когда независимые переменные (признаки) коррелируют друг с другом, т.е. имеет место быть мультиколлинеарность признаков.
* Регуляризация работает даже при полной мультиколлинеарности признаков.

Как уже отмечалось в начале, вы вряд ли будете использовать линейную регрессию и её модификации с регуляризацией в соревнованиях, но параметры отвечающие за регуляризацию присутствуют во многих популярных библиотеках и моделях машинного обучения (CatBoost, LightGBM, RandomForest, SVM, и многие другие). Надеемся, что освоив этот урок, вы будете более осознанно подходить к тюнингу того или иного параметра, а не просто рандомно перебирать цифры, пытаясь угадать заветную комбинацию 😁. 


## Полезные ссылки:
1. Отличный [курс от ODS](https://ods.ai/tracks/linear-models-spring22) про линейные модели и регуляризацию со всеми математическими выкладками.
2. Если хотите углубиться в математику (это поможет понять, как работает регуляризация), прочитайте главу 3.4 в книге [“Элементы статистического обучения”](https://hastie.su.domains/Papers/ESLII.pdf), написанной Треворой Хасти, Робертом Тибширани и Джеромом Фридманом. Роберт Тибширани  —  автор метода лассо-регрессии.
3. [Англоязычная статья](https://towardsdatascience.com/lasso-and-ridge-regression-an-intuitive-comparison-3ee415487d18), которая легла в основу этого ноутбука.